# Cria a Matriz Origem-Destino dos municípios e seus identificadores

In [1]:
import pandas as pd
import geopandas as gpd
from itertools import permutations
from geopy.distance import geodesic

In [2]:
centroide = gpd.read_file("data/shapefile/centroide/municipios_centroide.shp")

In [3]:
centroide.columns

Index(['CD_MUN', 'NM_MUN', 'SIGLA_UF', 'AREA_KM2', 'geometry'], dtype='object')

In [4]:
def gerar_base_distancias(centroide):
    # Função para calcular a distância entre dois pontos
    def calcular_distancia(coord1, coord2):
        if pd.notnull(coord1) and pd.notnull(coord2):
            # Extrair latitude e longitude do formato shapely Point
            coord1 = (coord1.y, coord1.x)  # (latitude, longitude)
            coord2 = (coord2.y, coord2.x)
            return geodesic(coord1, coord2).kilometers
        else:
            return None

    # Calcular o número de CD_MUN únicos
    cd_mun_unicos = centroide['CD_MUN'].nunique()

    # Gerar todas as combinações de pares onde a ordem importa (permutações)
    pares = pd.DataFrame(list(permutations(centroide['CD_MUN'], 2)), columns=['mun_origem', 'mun_destino'])

    # Fazer o merge para obter as coordenadas e SIGLA_UF de mun_origem e mun_destino
    pares = pares.merge(centroide[['CD_MUN', 'geometry']], left_on='mun_origem', right_on='CD_MUN').rename(columns={'geometry': 'geom_origem'})
    pares = pares.merge(centroide[['CD_MUN', 'geometry']], left_on='mun_destino', right_on='CD_MUN').rename(columns={'geometry': 'geom_destino'})

    # Inicializar variáveis de progresso
    valores_progresso = set()

    # Calcular as distâncias e exibir o progresso
    def calcular_e_mostrar_progresso(row):
        if row['mun_destino'] not in valores_progresso:  # Verificar se é um valor novo de mun_destino
            valores_progresso.add(row['mun_destino'])  # Adicionar valor único de mun_destino
            progresso_atual = len(valores_progresso)
            print(f"Progresso: {progresso_atual}/{cd_mun_unicos} CD_MUN únicos processados.")
        return calcular_distancia(row['geom_origem'], row['geom_destino'])

    # Aplicar a função de cálculo com exibição de progresso
    pares['distancia'] = pares.apply(lambda row: calcular_e_mostrar_progresso(row), axis=1)

    # Manter apenas as colunas relevantes
    return pares[['mun_origem', 'mun_destino', 'distancia']].reset_index(drop=True)

In [5]:
matriz_od_distancia = gerar_base_distancias(centroide)

Progresso: 1/5572 CD_MUN únicos processados.
Progresso: 2/5572 CD_MUN únicos processados.
Progresso: 3/5572 CD_MUN únicos processados.
Progresso: 4/5572 CD_MUN únicos processados.
Progresso: 5/5572 CD_MUN únicos processados.
Progresso: 6/5572 CD_MUN únicos processados.
Progresso: 7/5572 CD_MUN únicos processados.
Progresso: 8/5572 CD_MUN únicos processados.
Progresso: 9/5572 CD_MUN únicos processados.
Progresso: 10/5572 CD_MUN únicos processados.
Progresso: 11/5572 CD_MUN únicos processados.
Progresso: 12/5572 CD_MUN únicos processados.
Progresso: 13/5572 CD_MUN únicos processados.
Progresso: 14/5572 CD_MUN únicos processados.
Progresso: 15/5572 CD_MUN únicos processados.
Progresso: 16/5572 CD_MUN únicos processados.
Progresso: 17/5572 CD_MUN únicos processados.
Progresso: 18/5572 CD_MUN únicos processados.
Progresso: 19/5572 CD_MUN únicos processados.
Progresso: 20/5572 CD_MUN únicos processados.
Progresso: 21/5572 CD_MUN únicos processados.
Progresso: 22/5572 CD_MUN únicos processado

In [7]:
# Salvando a matriz_od_distancia como um arquivo CSV
output_path_csv = 'data/csv/matriz_od_distancia.csv'
matriz_od_distancia.to_csv(output_path_csv, index=False)